In [ ]:
import ijson
import numpy as np
import json
from decimal import Decimal
import json
import time
import torch
import warnings
import numpy as np
from IPython.display import clear_output
import pennylane as qml
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pennylane as qml
import numpy as np
import torch
from scipy.linalg import expm
import json
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np
from scipy.linalg import qr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np

In [ ]:


def cargar_datos_json(json_path, num_jets=10000, num_constituents=10):
    with open(json_path, 'r') as f:
        data = json.load(f)

    eventos = []
    for i, evento in enumerate(data[:num_jets]):
        # Extract jet kinematics
        jet_pt = evento.get('jet_pt', i)
        jet_eta = evento.get('jet_eta', i)
        jet_phi = evento.get('jet_phi', i)
        jet_mass = evento.get('jet_sdmass', i)
        jet_energy = evento.get('jet_energy', i)
        jet_tau1 = evento.get('jet_tau1', i)
        jet_tau2 = evento.get('jet_tau2', i)
        jet_tau3 = evento.get('jet_tau3', i)    
        jet_tau4 = evento.get('jet_tau4', i)

        jet_tau12 = jet_tau1 / jet_tau2 if jet_tau2 != 0 else 0
        jet_tau23 = jet_tau2 / jet_tau3 if jet_tau3 != 0 else 0
        jet_tau34 = jet_tau3 / jet_tau4 if jet_tau4 != 0 else 0

        # Extract constituents (particles)
        part_px = np.array(evento.get('part_px', []))
        part_py = np.array(evento.get('part_py', []))
        part_pz = np.array(evento.get('part_pz', []))
        part_energy = np.array(evento.get('part_energy', []))
        part_d0val = np.array(evento.get('part_d0val', []))
        part_dzval = np.array(evento.get('part_dzval', []))
        
        # Calculate pT, eta, phi, mass for each constituent
        pt = np.sqrt(part_px**2 + part_py**2)
        p_total = np.sqrt(part_px**2 + part_py**2 + part_pz**2)
        eta = 0.5 * np.log((p_total + part_pz) / (p_total - part_pz + 1e-8))  # Avoiding dividing by 0
        phi = np.arctan2(part_py, part_px)
        mass = np.sqrt(np.maximum(0, part_energy**2 - (part_px**2 + part_py**2 + part_pz**2)))
        
        # Seleccionar los num_constituents con mayor pt
        indices_ordenados = np.argsort(pt)[::-1][:num_constituents]
        
        top_constituents = []
        for idx in indices_ordenados:
            top_constituents.append({
                'pt': pt[idx],
                'eta': eta[idx],
                'phi': phi[idx],
                'px': part_px[idx],
                'py': part_py[idx],
                'pz': part_pz[idx],
                'mass': mass[idx],
                'energy': part_energy[idx],
                'd0': part_d0val[idx],
                'dz': part_dzval[idx]
            })
            
        eventos.append({
            'pt_jet': jet_pt,
            'eta_jet': jet_eta,
            'phi_jet': jet_phi,
            'mass_jet': jet_mass,
            'energy_jet': jet_energy,
            'tau1_jet': jet_tau1,
            'tau2_jet': jet_tau2,
            'tau3_jet': jet_tau3,
            'tau4_jet': jet_tau4,
            'tau12_jet': jet_tau12,
            'tau23_jet': jet_tau23,
            'tau34_jet': jet_tau34,
            'constituents': top_constituents
        })

    return eventos
# Ejemplo de uso
datos_HToBB = cargar_datos_json('./HToBB_120_flat.json', num_jets=10000, num_constituents=10)
datos_TTBar = cargar_datos_json('./TTBar_120_flat.json', num_jets=10000, num_constituents=10)
datos_WToqq = cargar_datos_json('./WToQQ_120_flat.json', num_jets=10000, num_constituents=10)
datos_QCD_simu_1 = cargar_datos_json('./ZJetsToNuNu_120_flat.json', num_jets=22500, num_constituents=10)
datos_QCD_simu_2 = cargar_datos_json('./ZJetsToNuNu_120_flat.json', num_jets=22500, num_constituents=10)
datos_QCD_simu_full = datos_QCD_simu_1 + datos_QCD_simu_2

In [ ]:
datos = np.array(datos_QCD_simu_full)

# Separar 10,000 para entrenamiento y 12,500 restantes
X_train, X_temp = train_test_split(
    datos, 
    train_size=10000, 
    random_state=42, 
    shuffle=True
)

# Separar 2,500 para validación y 10,000 para inferencia
X_val, rest = train_test_split(
    X_temp, 
    train_size=2500, 
    random_state=42, 
    shuffle=True
)

X_inf, rest = train_test_split(
    rest, 
    train_size=10000, 
    random_state=42, 
    shuffle=True
)

# Verificar tamaños
print(f"Entrenamiento: {len(X_train)}")
print(f"Validación: {len(X_val)}")
print(f"Inferencia: {len(X_inf)}")

In [ ]:


# Gell-Mann matrices (SU(3))
Lambda = {
    1: torch.tensor([[0, 1, 0],
                 [1, 0, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    2: torch.tensor([[0, -1j, 0],
                 [1j, 0, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    3: torch.tensor([[1, 0, 0],
                 [0, -1, 0],
                 [0, 0, 0]], dtype=torch.cdouble),

    4: torch.tensor([[0, 0, 1],
                 [0, 0, 0],
                 [1, 0, 0]], dtype=torch.cdouble),

    5: torch.tensor([[0, 0, -1j],
                 [0, 0, 0],
                 [1j, 0, 0]], dtype=torch.cdouble),

    6: torch.tensor([[0, 0, 0],
                 [0, 0, 1],
                 [0, 1, 0]], dtype=torch.cdouble),

    7: torch.tensor([[0, 0, 0],
                 [0, 0, -1j],
                 [0, 1j, 0]], dtype=torch.cdouble),

    8: (1/torch.sqrt(torch.tensor(3.0))) * torch.tensor([[1, 0, 0],
                                  [0, 1, 0],
                                  [0, 0, -2]], dtype=torch.cdouble),
    0: torch.tensor([[1, 0, 0],
                 [0, 1, 0],
                 [0, 0, 1]], dtype=torch.cdouble)
}

# Spin-1 rotation generators (SO(3) ⊂ SU(3))
Sigma = {
    1: (1 / torch.sqrt(torch.tensor(2.0))) * torch.tensor([
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0]
    ], dtype=torch.cdouble),

    2: (1 / torch.sqrt(torch.tensor(2.0))) * torch.tensor([
        [0, -1j, 0],
        [1j, 0, -1j],
        [0, 1j, 0]
    ], dtype=torch.cdouble),

    3: torch.tensor([
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, -1]
    ], dtype=torch.cdouble)
}

def TSWAP_matrix():
    tswap = np.zeros((9, 9), dtype=complex)
    for i in range(3):
        for j in range(3):
            ket = np.zeros(9)
            bra = np.zeros(9)
            ket[3*i + j] = 1   # |i⟩|j⟩
            bra[3*j + i] = 1   # |j⟩|i⟩
            tswap += np.outer(bra, ket)
    return tswap


def unitary_from_generator(generator_matrix, theta):
    if not torch.is_tensor(theta):
        theta = torch.tensor(theta, dtype=torch.cdouble)
    i = torch.tensor(1j, dtype=torch.cdouble)
    return Lambda[0] + (torch.cos(theta) - torch.tensor(1.0)) * generator_matrix @ generator_matrix + i * torch.sin(theta) * generator_matrix

def inicializing_qutrit_state(theta1, theta2, phi1, phi2):
    Gamma= 0
    a0 = 0
    a1 = 0
    a2 = 0

    Gamma = torch.sqrt(torch.tensor(2.0)) * (torch.tensor(3.0) + torch.cos(theta1)*torch.cos(theta2) + torch.sin(theta1)*torch.sin(theta2)*torch.cos(phi1 - phi2))**(torch.tensor(-0.5))

    a0 = (torch.sqrt(torch.tensor(2.0)) * torch.cos(theta1/2) * torch.cos(theta2/2)).item()
    a1 = (torch.exp(1j * phi1) * torch.sin(theta1/2) * torch.cos(theta2/2) + torch.cos(theta1/2) * torch.sin(theta2/2) * torch.exp(1j * phi2)).item()
    a2 = (torch.sqrt(torch.tensor(2.0)) * torch.exp(1j * (phi1 + phi2)) * torch.sin(theta1/2) * torch.sin(theta2/2)).item()

    state = Gamma * torch.tensor([a0, a1, a2], dtype=torch.cdouble)
    state = state / torch.linalg.norm(state)
    
    

    return state.detach().clone().numpy()



def unitary_from_state(psi):
    """
    psi: complex normalized vector of dimension 3 (qutrit)
    returns: 3x3 unitary whose first column is psi
    """
    psi = psi / np.linalg.norm(psi)  # por seguridad
    
    a1 = torch.tensor([0.555,0, 0.555], dtype=torch.cdouble)
    a2 = torch.tensor([0.555,0.555, 0], dtype=torch.cdouble)
    mat = np.column_stack([psi, a1 , a2])

    Q, R = qr(mat)
    phase = np.vdot(psi, Q[:,0])
    Q[:,0] = Q[:,0] * (phase/abs(phase)).conj()
    
    return Q



# **MODEL DEFINITION**

In [ ]:
# Parameters
from autoray import do


num_particles = 4
num_latent = 1
num_ref = num_particles - num_latent
num_trash = num_ref
wires = list(range(num_particles + num_ref + 1))  # +1 ancilla
ancilla = wires[-1]
dev = qml.device("default.qutrit", wires=wires)  

latent_wire = 0
trash_wires = wires[1:num_particles]
ref_wires = wires[num_particles:-1]

# Encoding functions
def f(w): return 1 + (2 * np.pi / (1 + torch.exp(-w)))
def phi_circuit(w, phi, phi_jet, pt, pt_jet): return f(w) * (pt / pt_jet) * (phi - phi_jet)
def theta_circuit(w, eta, eta_jet, pt, pt_jet): return f(w) * (pt / pt_jet) * (eta - eta_jet)
def mass_circuit(w, mass, mass_jet, pt, pt_jet):  return  f(w) * (pt / pt_jet) * (mass - mass_jet)
def energy_circuit(w, energy, energy_jet, pt, pt_jet): return f(w) * (pt / pt_jet) *  (energy - energy_jet)
def d0_circuit(w, d0, pt, pt_jet): return f(w) * (pt / pt_jet) * (d0)
def dz_circuit(w, dz, pt, pt_jet): return f(w) * (pt / pt_jet) * (dz)
def tau12_circuit(w, tau12, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau12)
def tau23_circuit(w, tau23, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau23)
def tau34_circuit(w, tau34, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau34)
def tau1_circuit(w, tau1, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau1)
def tau2_circuit(w, tau2, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau2)
def tau3_circuit(w, tau3, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau3)
def tau4_circuit(w, tau4, pt, pt_jet): return f(w) * (pt / pt_jet) * (tau4)

# Encoding for qutrits
def encode_1p1q_qutrit(jets, w, unitaries):

    pt_jet = jets['pt_jet']
    eta_jet = jets['eta_jet']
    phi_jet = jets['phi_jet']
    mass_jet = jets['mass_jet']
    energy_jet = jets['energy_jet']
    #tau12_jet = jets['tau12_jet']
    #tau23_jet = jets['tau23_jet']
    #tau34_jet = jets['tau34_jet']
    constituents = jets['constituents']
        
    for i in range(num_particles):
        c = constituents[i]
        theta = theta_circuit(w, c['eta'], eta_jet, c['pt'], pt_jet)
        phi = phi_circuit(w, c['phi'], phi_jet, c['pt'], pt_jet)
        mass = mass_circuit(w, c['mass'], mass_jet, c['pt'], pt_jet)
        energy = energy_circuit(w, c['energy'], energy_jet, c['pt'], pt_jet)
        d0 = d0_circuit(w, c['d0'], c['pt'], pt_jet)
        dz = dz_circuit(w, c['dz'], c['pt'], pt_jet)
        #tau12 = tau12_circuit(w,  tau12_jet ,c['pt'], pt_jet)
        #tau23 = tau23_circuit(w,  tau23_jet, c['pt'], pt_jet)
        #tau34 = tau34_circuit(w,  tau34_jet, c['pt'], pt_jet)
        #tau1 = tau1_circuit(w, jets['tau1_jet'], c['pt'], pt_jet)
        #tau2 = tau2_circuit(w, jets['tau2_jet'], c['pt'], pt_jet)
        #tau3 = tau3_circuit(w, jets['tau3_jet'], c['pt'], pt_jet)
        #tau4 = tau4_circuit(w, jets['tau4_jet'], c['pt'], pt_jet)

        initial_state = inicializing_qutrit_state( theta, phi, d0, dz)
        u = unitary_from_state(initial_state)
        unitaries.append(u)
        qml.QutritUnitary(u, wires=i)    

def decode_from_unitaries(unitaries):
    for i, u in enumerate(unitaries):
        qml.QutritUnitary(u.conj().T, wires=i)

# Variational layer for qutrits (ENCODER)
def variational_layer_qutrit(theta_i, phi_i, w_i, num_layers):
    for layer in range(num_layers):
        for i in range(num_particles):
            for j in range(i + 1, num_particles):
                qml.TAdd(wires=[i, j])
        for i in range(num_particles):

            RX = unitary_from_generator(Sigma[1], phi_i[layer, i])
            RY = unitary_from_generator(Sigma[2], theta_i[layer, i])
            RZ = unitary_from_generator(Sigma[3], w_i[layer, i])
    
            qml.QutritUnitary(RX, wires=i)
            qml.QutritUnitary(RZ, wires=i)
            qml.QutritUnitary(RY, wires=i)


# DECODER (hermite conjugate encoder)
def variational_layer_qutrit_dagger(theta_i, phi_i, w_i, num_layers):
    # Recorremos las capas en orden inverso
    for layer in reversed(range(num_layers)):

        # Invertir las rotaciones locales primero (orden inverso y adjunto)
        for i in reversed(range(num_particles)):
            RX = unitary_from_generator(Sigma[1], phi_i[layer, i])
            RY = unitary_from_generator(Sigma[2], theta_i[layer, i])
            RZ = unitary_from_generator(Sigma[3], w_i[layer, i])

            # Aplicar el inverso en orden opuesto
            qml.QutritUnitary(RY.conj().T, wires=i)
            qml.QutritUnitary(RZ.conj().T, wires=i)
            qml.QutritUnitary(RX.conj().T, wires=i)

        # Ahora invertir los TAdd (mismo gate, pero orden inverso)
        for i in reversed(range(num_particles)):
            for j in reversed(range(i + 1, num_particles)):
                qml.TAdd(wires=[i, j])

# QAE circuit for qutrits
@qml.qnode(dev, interface="torch", diff_method="backprop")
def qae_circuit_qutrit(jets, w, theta_i, phi_i, w_i, num_layers):
    unitaries = []
    encode_1p1q_qutrit(jets, w, unitaries)
    variational_layer_qutrit(theta_i, phi_i, w_i, num_layers)
    tswap = TSWAP_matrix()

    for trash_wire, ref_wire in zip(trash_wires, ref_wires):
        qml.THadamard(wires=ancilla, subspace=None) #With none they apply the generalized version
        qml.ControlledQutritUnitary(tswap, control_wires=ancilla, wires=[trash_wire, ref_wire])
        qml.THadamard(wires=ancilla, subspace=None)
    
    return qml.probs(wires=ancilla)

# Cost function
def cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i, num_layers):
    prob_0 = qae_circuit_qutrit(jet, w, theta_i, phi_i, w_i, num_layers)[0]
    fidelity = prob_0
    return -fidelity, fidelity.item()


In [43]:
fil_100_back = []
fil_100_HToBB = []
fil_100_WToQQ = []
fil_100_TTBar = []

for i in range(100):

    w = torch.tensor(1.0, requires_grad=True)
    num_layers = 1 # Number of variational layers
    theta_i = (torch.rand(num_layers, num_particles) * 2 * torch.pi).requires_grad_(True)
    phi_i   = (torch.rand(num_layers, num_particles) * 2 * torch.pi).requires_grad_(True)
    w_i     = (torch.rand(num_layers, num_particles) * 2 * torch.pi).requires_grad_(True)
    optimizer = torch.optim.Adam(
        [w, theta_i, phi_i, w_i],
        lr=5e-2,              
        betas=(0.5, 0.999),
        eps=1e-08,
        weight_decay=0.0,    
        amsgrad=True          
    )
    num_epochs = 1
    all_fidelities = []
    event_fidelities = []  # List to store event fidelities

    for epoch in range(num_epochs):
        total_loss = 0.0
        epoch_fidelities = []
        avg_fidelity = 0.0
        avg_loss = 0.0
        
        for jet in X_train:
            if len(jet['constituents']) < num_particles:
                continue
        
            loss, fidelity = cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i,  num_layers)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            epoch_fidelities.append(fidelity)
            event_fidelities.append(fidelity * 100)  # in %

        avg_loss = total_loss / len(epoch_fidelities)
        avg_fidelity = np.mean(epoch_fidelities) * 100
        all_fidelities.append(avg_fidelity)


    event_fidelities_back = []
    event_fidelities_HToBB = []
    event_fidelities_WToQQ = []
    event_fidelities_TTBar = []
    fidelidades = []
    etiquetas = []

    
    for jet in X_inf:
        if len(jet['constituents']) < num_particles:
            continue
        _, fidelity = cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i, num_layers)
        event_fidelities_back.append(fidelity * 100) 
        fidelidades.append(fidelity)
        etiquetas.append(0)

    

    inicio = time.time()
    for jet in datos_HToBB:
        if len(jet['constituents']) < num_particles:
            continue
        _, fidelity = cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i, num_layers)
        event_fidelities_HToBB.append(fidelity * 100) 
        fidelidades.append(fidelity)
        etiquetas.append(1)

    

    for jet in datos_TTBar:
        if len(jet['constituents']) < num_particles:
            continue
        _, fidelity = cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i, num_layers)
        event_fidelities_TTBar.append(fidelity * 100) 
        fidelidades.append(fidelity)
        etiquetas.append(1)

    

    for jet in datos_WToqq:
        if len(jet['constituents']) < num_particles:
            continue
        _, fidelity = cost_function_with_fidelity_qutrit(jet, w, theta_i, phi_i, w_i, num_layers)
        event_fidelities_WToQQ.append(fidelity * 100) 
        fidelidades.append(fidelity)
        etiquetas.append(1)

    

    fil_100_back.append(event_fidelities_back)
    fil_100_HToBB.append(event_fidelities_HToBB)
    fil_100_WToQQ.append(event_fidelities_WToQQ)
    fil_100_TTBar.append(event_fidelities_TTBar)

    print("Fin etapa ", i+1)


Fin etapa  1
Fin etapa  2
Fin etapa  3
Fin etapa  4
Fin etapa  5
Fin etapa  6
Fin etapa  7
Fin etapa  8
Fin etapa  9
Fin etapa  10
Fin etapa  11
Fin etapa  12
Fin etapa  13
Fin etapa  14
Fin etapa  15
Fin etapa  16
Fin etapa  17
Fin etapa  18
Fin etapa  19
Fin etapa  20
Fin etapa  21
Fin etapa  22
Fin etapa  23
Fin etapa  24
Fin etapa  25
Fin etapa  26
Fin etapa  27
Fin etapa  28
Fin etapa  29
Fin etapa  30
Fin etapa  31
Fin etapa  32
Fin etapa  33
Fin etapa  34
Fin etapa  35
Fin etapa  36
Fin etapa  37
Fin etapa  38
Fin etapa  39
Fin etapa  40
Fin etapa  41
Fin etapa  42
Fin etapa  43
Fin etapa  44
Fin etapa  45
Fin etapa  46
Fin etapa  47
Fin etapa  48
Fin etapa  49
Fin etapa  50
Fin etapa  51
Fin etapa  52
Fin etapa  53
Fin etapa  54
Fin etapa  55
Fin etapa  56
Fin etapa  57
Fin etapa  58
Fin etapa  59
Fin etapa  60
Fin etapa  61
Fin etapa  62
Fin etapa  63
Fin etapa  64
Fin etapa  65
Fin etapa  66
Fin etapa  67
Fin etapa  68
Fin etapa  69
Fin etapa  70
Fin etapa  71
Fin etapa  72
F

In [ ]:
fil_100_back_2=fil_100_back,
fil_100_HToBB_2=fil_100_HToBB,
fil_100_WToQQ_2=fil_100_WToQQ,
fil_100_TTBar_2=fil_100_TTBar

np.savez(
    'fidelidades_100.npz',
    fil_100_back_S=fil_100_back_2,
    fil_100_HToBB_S=fil_100_HToBB_2,
    fil_100_WToQQ_S=fil_100_WToQQ_2,
    fil_100_TTBar_S=fil_100_TTBar_2
)
print("Guardado como fidelidades_100.npz")

In [ ]:
data = np.load('./fidelidades_100.npz', allow_pickle=True)
fil_100_back_3= data['fil_100_back_S']
fil_100_HToBB_3 = data['fil_100_HToBB_S']
fil_100_WToQQ_3 = data['fil_100_WToQQ_S']
fil_100_TTBar_3 = data['fil_100_TTBar_S']

fil_100_back_3 = fil_100_back_3.squeeze()
fil_100_HToBB_3 = fil_100_HToBB_3.squeeze()
fil_100_WToQQ_3 = fil_100_WToQQ_3.squeeze()
fil_100_TTBar_3 = fil_100_TTBar_3.squeeze()



In [ ]:


means_HToBB = []
means_WToQQ = []
means_TTBar = []
means_back = []

median_HToBB = []
median_WToQQ = []
median_TTBar = []
median_back = []

for i in range(100):
    means_back.append(np.mean(fil_100_back_3[i]))
    means_HToBB.append(np.mean(fil_100_HToBB_3[i]))
    means_WToQQ.append(np.mean(fil_100_WToQQ_3[i]))
    means_TTBar.append(np.mean(fil_100_TTBar_3[i]))

    median_back.append(np.median(fil_100_back_3[i]))
    median_HToBB.append(np.median(fil_100_HToBB_3[i]))
    median_WToQQ.append(np.median(fil_100_WToQQ_3[i]))
    median_TTBar.append(np.median(fil_100_TTBar_3[i]))

In [241]:
print("Anomaly Score BACKGROUND:", 1 - np.min(means_back)/100)

print("Estadísticas de las fidelidades medias para HToBB:")
print("Media:", np.mean(means_HToBB))
print("Mediana:", np.median(median_HToBB))
print("Mínimo:", np.min(means_HToBB))
print("Máximo:", np.max(means_HToBB))
print("Desviación estándar:", np.std(means_HToBB))
print("Anomaly Score:", 1 - np.min(means_HToBB)/100)
print()
print("Estadísticas de las fidelidades medias para WToQQ:")
print("Media:", np.mean(means_WToQQ))
print("Mediana:", np.median(median_WToQQ))
print("Mínimo:", np.min(means_WToQQ))
print("Máximo:", np.max(means_WToQQ))
print("Desviación estándar:", np.std(means_WToQQ))
print("Anomaly Score:", 1 - np.min(means_WToQQ)/100)
print()
print("Estadísticas de las fidelidades medias para TTBar:")
print("Media:", np.mean(means_TTBar))
print("Mediana:", np.median(median_TTBar))
print("Mínimo:", np.min(means_TTBar))
print("Máximo:", np.max(means_TTBar))   
print("Desviación estándar:", np.std(means_TTBar))
print("Anomaly Score:", 1 - np.min(means_TTBar)/100)


print("Difereferenicas entre medias")
print("HToBB - WToQQ:", np.mean(means_HToBB) - np.mean(means_WToQQ))
print("HToBB - TTBar:", np.mean(means_HToBB) - np.mean(means_TTBar))
print("WToQQ - TTBar:", np.mean(means_WToQQ) - np.mean(means_TTBar))

print("Difereferenicas entre medianas")
print("HToBB - WToQQ:", np.median(median_HToBB) - np.median(median_WToQQ))
print("HToBB - TTBar:", np.median(median_HToBB) - np.median(median_TTBar))
print("WToQQ - TTBar:", np.median(median_WToQQ) - np.median(median_TTBar))


Anomaly Score BACKGROUND: 0.010935291821573379
Estadísticas de las fidelidades medias para HToBB:
Media: 97.9631863932231
Mediana: 99.80948141693219
Mínimo: 97.96314968539224
Máximo: 97.9632176004691
Desviación estándar: 1.3280851160762936e-05
Anomaly Score: 0.020368503146077632

Estadísticas de las fidelidades medias para WToQQ:
Media: 98.81728877666805
Mediana: 99.86475336791814
Mínimo: 98.81723730903295
Máximo: 98.81731600130546
Desviación estándar: 1.4237139833855767e-05
Anomaly Score: 0.011827626909670541

Estadísticas de las fidelidades medias para TTBar:
Media: 97.25351168519006
Mediana: 99.60967275453382
Mínimo: 97.25347624060994
Máximo: 97.25353830731318
Desviación estándar: 1.3109564762407157e-05
Anomaly Score: 0.027465237593900627
Difereferenicas entre medias
HToBB - WToQQ: -0.8541023834449533
HToBB - TTBar: 0.7096747080330346
WToQQ - TTBar: 1.5637770914779878
Difereferenicas entre medianas
HToBB - WToQQ: -0.05527195098595428
HToBB - TTBar: 0.19980866239836814
WToQQ - TTBar:

In [ ]:
data = np.load('./fidelidades_100.npz', allow_pickle=True)
fil_100_back_4auc= data['fil_100_back_S']
fil_100_HToBB_4auc = data['fil_100_HToBB_S']
fil_100_WToQQ_4auc = data['fil_100_WToQQ_S']
fil_100_TTBar_4auc = data['fil_100_TTBar_S']

In [ ]:


auc_HToBB_list = []
auc_TTBar_list = []
auc_WToQQ_list = []

fil_100_back_4auc = fil_100_back_4auc.squeeze()
fil_100_HToBB_4auc = fil_100_HToBB_4auc.squeeze()
fil_100_WToQQ_4auc = fil_100_WToQQ_4auc.squeeze()
fil_100_TTBar_4auc = fil_100_TTBar_4auc.squeeze()

for i in range(100):
    event_fidelities_back = fil_100_back_4auc[i]
    event_fidelities_HToBB = fil_100_HToBB_4auc[i]
    event_fidelities_TTBar = fil_100_TTBar_4auc[i]
    event_fidelities_WToQQ = fil_100_WToQQ_4auc[i]

    # --- Anomaly scores (1 - Fidelity) ---
    anomaly_scores_back = 1 - np.array(event_fidelities_back).ravel()
    anomaly_scores_HToBB = 1 - np.array(event_fidelities_HToBB).ravel()
    anomaly_scores_TTBar = 1 - np.array(event_fidelities_TTBar).ravel()
    anomaly_scores_WToQQ = 1 - np.array(event_fidelities_WToQQ).ravel() 

    # --- Calcular AUC ---
    auc_HToBB = roc_auc_score(
        np.concatenate([np.zeros_like(anomaly_scores_back), np.ones_like(anomaly_scores_HToBB)]),
        np.concatenate([anomaly_scores_back, anomaly_scores_HToBB])
    )

    auc_TTBar = roc_auc_score(
        np.concatenate([np.zeros_like(anomaly_scores_back), np.ones_like(anomaly_scores_TTBar)]),
        np.concatenate([anomaly_scores_back, anomaly_scores_TTBar])
    )

    auc_WToQQ = roc_auc_score(
        np.concatenate([np.zeros_like(anomaly_scores_back), np.ones_like(anomaly_scores_WToQQ)]),
        np.concatenate([anomaly_scores_back, anomaly_scores_WToQQ])
    )

    auc_HToBB_list.append(auc_HToBB)
    auc_TTBar_list.append(auc_TTBar)
    auc_WToQQ_list.append(auc_WToQQ)

#medias de AUCs
mean_auc_HToBB = np.mean(auc_HToBB_list)
mean_auc_TTBar = np.mean(auc_TTBar_list)
mean_auc_WToQQ = np.mean(auc_WToQQ_list)


print("Mean AUC HToBB:", mean_auc_HToBB)
print("Mean AUC TTBar:", mean_auc_TTBar)
print("Mean AUC WToQQ:", mean_auc_WToQQ)

Mean AUC HToBB: 0.7003041905
Mean AUC TTBar: 0.7890799033999998
Mean AUC WToQQ: 0.6482334243000001


In [245]:
print("Estadísticas de las AUC medias para HToBB:")
print("Media:", np.mean(auc_HToBB_list))
print("Mínimo:", np.min(auc_HToBB_list))
print("Máximo:", np.max(auc_HToBB_list))
print("Desviación estándar:", np.std(auc_HToBB_list))
print()
print("Estadísticas de las AUC  medias para WToQQ:")
print("Media:", np.mean(auc_WToQQ_list))
print("Mínimo:", np.min(auc_WToQQ_list))
print("Máximo:", np.max(auc_WToQQ_list))
print("Desviación estándar:", np.std(auc_WToQQ_list))
print()
print("Estadísticas de las AUC  medias para TTBar:")
print("Media:", np.mean(auc_TTBar_list))
print("Mínimo:", np.min(auc_TTBar_list))
print("Máximo:", np.max(auc_TTBar_list))   
print("Desviación estándar:", np.std(auc_TTBar_list))

Estadísticas de las AUC medias para HToBB:
Media: 0.7003041905
Mínimo: 0.7002975
Máximo: 0.7003258
Desviación estándar: 3.6216687797182165e-06

Estadísticas de las AUC  medias para WToQQ:
Media: 0.6482334243000001
Mínimo: 0.64821266
Máximo: 0.64824222
Desviación estándar: 3.3007757436707747e-06

Estadísticas de las AUC  medias para TTBar:
Media: 0.7890799033999998
Mínimo: 0.78905882
Máximo: 0.78908506
Desviación estándar: 3.166291275285535e-06
